### Main Notebook for visualisation 

Setup for required libraries:

In [1]:
%pip install numpy pandas matplotlib keras sklearn tensorflow


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Importing data for DNN training (using Pandas):

In [2]:
import pandas as pd
import numpy as np

file_path = 'dtag/grid/4_agents/4/velocity_metrics.csv'
file_path2 = 'dtag/grid/4_agents/4/idleness.csv'
file_path3 = 'dtag/grid/4_agents/4/vulnerabilities.csv'

df_vel = pd.read_csv(file_path, sep=';', header=None)
df_idle = pd.read_csv(file_path2, sep=';',header=None)
df_vuln = pd.read_csv(file_path3, sep=';',header=None)
df_vel


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,61.0,0.311461,-0.122199,0.094301,4.720490,0.536859,2.107673,-0.575862,-0.741558,-0.548367,...,0.065813,1.121916,4.608600,1.761530,0.657547,-0.250035,0.907704,1.588041,1.070683,0.620057
1,62.0,0.004346,-0.064677,0.006699,0.870710,-0.133432,0.642107,-0.167152,-0.091446,-0.036904,...,-0.065078,0.223056,0.875101,0.097307,0.035532,-0.286550,0.457349,0.258541,0.088231,0.041765
2,63.0,0.028377,-0.054144,-0.008344,0.959266,-0.086637,1.163336,-0.111157,-0.058914,-0.025746,...,-0.073076,0.098002,1.021483,0.087074,0.040379,-0.197092,0.565688,0.201149,0.077965,0.041836
3,64.0,0.013608,-0.011577,-0.000097,0.170876,-0.018823,0.992261,-0.026125,0.002972,0.017747,...,-0.085974,0.062958,0.027213,0.055801,0.036591,-0.159622,1.144706,0.137316,0.066731,0.041929
4,65.0,-0.022561,-0.002280,0.009880,-0.503325,-0.060368,-0.010254,0.021521,0.060407,0.116396,...,-0.038173,0.026926,0.395588,0.145119,0.087466,-0.128190,0.450434,0.187478,0.126532,0.083793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21482,21610.6,-0.168537,-0.097641,-0.002175,-0.002604,-0.003031,0.846062,0.602596,0.016118,0.007458,...,0.080836,0.018233,-0.016916,-0.014659,-0.006684,0.081907,-0.168221,0.008621,-0.011392,-0.007831
21483,21611.6,-0.094388,-0.073272,0.008076,0.020187,0.021383,0.840653,-0.438621,0.065774,0.186821,...,0.104761,-0.075051,-0.052886,-0.057357,-0.029184,0.188299,-0.594788,-0.112083,-0.076874,-0.038386
21484,21612.6,-0.101074,-0.138166,0.077589,0.026911,0.109963,0.396401,-0.729262,0.134231,-0.583373,...,0.095113,-0.064345,-0.002854,-0.009607,0.031245,0.247693,-0.404475,-0.092880,-0.057226,0.009822
21485,21613.6,-0.099175,-0.192131,0.061002,-0.010641,0.150501,1.106362,-0.529410,0.154327,-0.443232,...,0.081831,-0.129329,-0.063124,-0.007774,0.030593,0.366857,-0.321969,-0.104097,-0.071536,0.009258


Melting Data so Index is each node:

In [3]:

df_melted_vel = df_vel.melt(id_vars=[0], var_name='Node', value_name='Velocity')
df_melted_vel.set_index('Node', inplace=True)
df_melted_idle = df_idle.melt(id_vars=[0], var_name='Node', value_name='Idleness')
df_melted_idle.set_index('Node', inplace=True)
df_melted_vuln = df_vuln.melt(id_vars=[0], var_name='Node', value_name='Vulnerability')
df_melted_vuln.set_index('Node', inplace=True)


# Renaming columns:
df_melted_vel.columns = ['Timestep', 'Velocity']
df_melted_idle.columns = ['Timestep', 'Idleness']
df_melted_vuln.columns = ['Timestep', 'Vulnerability']

df_melted_vel

,Timestep,Velocity
Node,,
1,61.0,0.311461
1,62.0,0.004346
1,63.0,0.028377
1,64.0,0.013608
1,65.0,-0.022561
...,...,...
25,21610.6,-0.007831
25,21611.6,-0.038386
25,21612.6,0.009822


Merging data into datasets with 1 row for each node:

In [4]:
merged_df_vel = df_melted_vel.groupby(df_melted_vel.index).agg(list)
merged_df_idle = df_melted_idle.groupby(df_melted_idle.index).agg(list)
merged_df_vuln = df_melted_vuln.groupby(df_melted_vuln.index).agg(list)

merged_df_vel

,Timestep,Velocity
Node,,
1,"[61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68....","[0.31146059840476625, 0.004346260037849228, 0...."
2,"[61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68....","[-0.12219872413861624, -0.06467660021957267, -..."
3,"[61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68....","[0.0943012156517324, 0.006698620307105309, -0...."
4,"[61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68....","[4.720489587299716, 0.8707102570527958, 0.9592..."
5,"[61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68....","[0.536859192672703, -0.13343201430559204, -0.0..."
6,"[61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68....","[2.1076733678809645, 0.6421072261151488, 1.163..."
7,"[61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68....","[-0.5758618945020709, -0.16715208452558403, -0..."
8,"[61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68....","[-0.7415582120250857, -0.0914460981911138, -0...."
9,"[61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68....","[-0.5483670209246784, -0.03690398634069965, -0..."


In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, concatenate


2023-10-19 10:34:26.193001: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Reshaped data for setting up Dense input layers (Had problems with datatype, it works but cba to tidy it up):

In [18]:
# idle_input = merged_df_idle['Idleness'].to_numpy()
# vel_input = merged_df_vel['Velocity'].to_numpy()
# vuln_output = merged_df_vuln['Vulnerability'].to_numpy()

idle_input = df_idle.T.to_numpy()
vel_input = df_vel.T.to_numpy()
vuln_output = df_vuln.T.to_numpy()

input_shape = idle_input.shape
input_layer1 = Input(shape=input_shape)
input_layer2 = Input(shape=input_shape)
submodel1 = Dense(4, activation='relu')(input_layer1)
submodel2 = Dense(4, activation='relu')(input_layer2)
merged = concatenate([submodel1, submodel2])


In [16]:
idle_input
vel_input.reshape(1,26)

<KerasTensor: shape=(None, 1, 26, 21487) dtype=float32 (created by layer 'input_5')>

Setting up extra layers and model:

In [13]:
#Extra layers and output:
x = Dense(4, activation='relu')(merged)
output_layer = Dense(1, activation='sigmoid')(x) 

# Create the final model
model = keras.models.Model(inputs=[input_layer1, input_layer2] , outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit([vel_input, idle_input], vuln_output, epochs=10, batch_size=32)

Epoch 1/10


ValueError: in user code:

    File "/Users/alexbott/4th Year local/Intelligent-Intruder/.venv/lib/python3.8/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/Users/alexbott/4th Year local/Intelligent-Intruder/.venv/lib/python3.8/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/alexbott/4th Year local/Intelligent-Intruder/.venv/lib/python3.8/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/Users/alexbott/4th Year local/Intelligent-Intruder/.venv/lib/python3.8/site-packages/keras/src/engine/training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "/Users/alexbott/4th Year local/Intelligent-Intruder/.venv/lib/python3.8/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_